| Swiftest TAP Workshop | Oct. 21, 2025 | Prof. David Minton |
| --------------------- | ------------- | ------------------ |


> Notes: This tutorial is based on the Detaioled simulation setup from the Swiftest User Guide [documentation page](https://swiftest.readthedocs.io/en/latest/user-guide/detailed-simulation-setup.html)

Here I will demonstrate some of the more advanced capabilities of Swiftest. 

Start with importing Swiftest and Numpy.

In [1]:
import swiftest
import numpy as np
import xarray as xr
xr.set_options(display_max_rows=50)
xr.set_options(use_new_combine_kwarg_defaults=True)

## Initial Simulation Setup 


Create a Swiftest Simulation object and clean the simulation directory of any previous Swiftest objects, if any.
Outputs are stored in the ``./simdata`` directory by default. 

An optional argument can be passed to specify the simulation directory 

```python
sim = swiftest.Simulation(simdir='/path/to/simdata')
```

The argument to `simdir` can either be a relative path or an absolute path.  Let's set a custom relative path that will put our data in a local folder called `tap_example`:

In [2]:
sim = swiftest.Simulation(simdir="tap_example")

Reading Swiftest file /Users/daminton/work/Talks/2025-10-TAP_University_of_Arizona/Swiftest_2025_TAP_Workshop/detailed_simulation/tap_example/param.in
codename                         Swiftest
gmtiny                           1.0919433586029915e-07 DU^3 / TU^2 
mtiny                            2.766029318728523e-09 MU
t0                               0.0 TU
tstart                           0.0 TU
tstop                            1000.0 TU
dt                               0.01 TU
istep_out                        100 
dump_cadence                     0 
tstep_out                        1.0 TU
init_cond_file_name              init_cond.nc
init_cond_file_type              NETCDF_DOUBLE
init_cond_format                 XV
output_file_type                 NETCDF_DOUBLE
output_file_name                 data.nc
output_format                    XVEL
restart                          REPLACE
rmin                             0.05 DU
rmax                             10000.0 DU
qmin_coord           

Now that we have a simulation object set up (with default parameters), we can add bodies to the simulation.  The most massive body in the simulation is taken as the central body.

## Solar System Bodies


We can add solar system bodies to the simulation using the :meth:`add_solar_system_body <swiftest.Simulation.add_solar_system_body>` method. 
This method uses JPL Horizons to extract the parameters of a particular body given a name.

In [3]:
# Add the modern planets and the Sun using the JPL Horizons Database.
sim.add_solar_system_body(["Sun","Mercury","Venus","Earth","Mars","Jupiter","Saturn","Uranus","Neptune"])

Adding solar system bodies by name: Sun, Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, Neptune
Creating the Sun as a central body
Fetching ephemerides data for Mercury from JPL/Horizons
Found matching body: Mercury Barycenter (1)
    Alternate matching bodies:
    Mercury (199)
Found ephemerides data for Mercury Barycenter (1) from JPL/Horizons
Physical properties found for Mercury Barycenter (199)
Fetching ephemerides data for Venus from JPL/Horizons
Found matching body: Venus Barycenter (2)
    Alternate matching bodies:
    Venus (299)
Found ephemerides data for Venus Barycenter (2) from JPL/Horizons
Physical properties found for Venus Barycenter (299)
Fetching ephemerides data for Earth from JPL/Horizons
Found matching body: Earth-Moon Barycenter (3)
    Alternate matching bodies:
    Earth (399)
Found ephemerides data for Earth-Moon Barycenter (3) from JPL/Horizons
Found matching body: Earth (399) (399)
Physical properties found for Earth (399)
Combining mass of Earth and 

We can add other small bodies too. 

In [4]:
# Add in some main belt asteroids
sim.add_solar_system_body(name=["Ceres","Vesta","Pallas","Hygiea"],id_type="smallbody")

# Add in some big KBOs
sim.add_solar_system_body(name=["Pluto","Eris","Haumea","Quaoar"])

# Add in some Centaurs
sim.add_solar_system_body(name=["Chiron","Chariklo"])

Adding solar system bodies by name: Ceres, Vesta, Pallas, Hygiea
Fetching ephemerides data for Ceres from JPL/Horizons
Found matching body: 1 Ceres (A801 AA) (Ceres)
Found ephemerides data for 1 Ceres (A801 AA) (Ceres) from JPL/Horizons
Physical properties found for 1 Ceres (A801 AA)
Fetching ephemerides data for Vesta from JPL/Horizons
Found matching body: 4 Vesta (A807 FA) (Vesta)
Found ephemerides data for 4 Vesta (A807 FA) (Vesta) from JPL/Horizons
Physical properties found for 4 Vesta (A807 FA)
Fetching ephemerides data for Pallas from JPL/Horizons
Found matching body: 2 Pallas (A802 FA) (Pallas)
Found ephemerides data for 2 Pallas (A802 FA) (Pallas) from JPL/Horizons
Physical properties found for 2 Pallas (A802 FA)
Fetching ephemerides data for Hygiea from JPL/Horizons
Found matching body: 10 Hygiea (A849 GA) (Hygiea)
Found ephemerides data for 10 Hygiea (A849 GA) (Hygiea) from JPL/Horizons
Physical properties found for 10 Hygiea (A849 GA)
rmin                             0.00465

> Note: The `add_solar_system_body>` method is designed for ease of use. If you pass the `name` argument alone, it will make a "best guess" as to which body to retrieve, using the `astroquery.jplhorizons` module. If you want more control over which body to pass to JPL Horizons, you can supply the optional argument `ephemeris_id` in addition to `name`. The string argument passed to `name` is then used internally by Swiftest to identify the body, but the query to JPL Horizons is made with `ephemeris_id`. Therefore the following two calls are equivalent

> Note: The arguments `name="Earth"` and `name="Pluto"` are handled as special cases in `add_solar_system_body` due to their unusually massive satellites. When "Earth" (or "Pluto") is requested, then the mass the Moon (Charon) is added to the body mass and the initial conditions are set to the Earth-Moon (Pluto-Charon) barycenter. If you wish to instead request the planet directly, you should pass `ephemeris_id` instead. 

## User-defined bodies

You can add a user-defined body with arbitrary initial conditions using using :meth:`sim.add_body <swiftest.Simulation.add_body>`. This method contains a number of optional arguments, and different combinations of arguments can result in different kinds of bodies. 

- id: This is a unique, positive integer id for the body. Usually you would not pass this argument, as an id will be automatically assigned in the order in which it was added, and the central body is always assigned to be id 0.

- name: This is a unique, string name for the body. If you do not pass this, then the name will be set to "Body{id}"

- rh,vh: These are the position and velocity vectors of the body in Cartesian coordinates. These are used to set the initial conditions for the body when the Simulation is set to `init_cond_format="XV"` (or the equivalent `param["IN_FORM"] = "XV"`).

- a,e,inc,capom,omega,capm: These are used to set the initial osculating orbital elements for the body when the Simulation is set to `init_cond_format="EL"` (or the equivalent `param["IN_FORM"] = "EL"`). 


We will randomize the initial conditions and therefore import the Numpy random module

In [5]:
from numpy.random import default_rng
rng = default_rng(seed=123)

Starting with **massive bodies:** 

In [6]:
npl = 5 # number of massive bodies
density_pl  = 3000.0 / (sim.param['MU2KG'] / sim.param['DU2M'] ** 3)
name_pl     = ["SemiBody_01", "SemiBody_02", "SemiBody_03", "SemiBody_04", "SemiBody_05"]

M_pl        = np.array([6e20, 8e20, 1e21, 3e21, 5e21]) * sim.KG2MU # mass in simulation units
R_pl        = np.full(npl, (3 * M_pl/ (4 * np.pi * density_pl)) ** (1.0 / 3.0)) # radius
Ip_pl       = np.full((npl,3),0.4,) # moment of inertia
rot_pl      = np.zeros((npl,3)) # initial rotation vector in degrees/TU
mtiny       = 1.1 * np.max(M_pl) # threshold mass for semi-interacting bodies in SyMBA.

Depending on the simulation parameters, we can add bodies with Orbital Elements or Cartesian Coordinates.

Orbital Elements
-------------------

Initialize orbital elements and then add the bodies.

In [7]:
a_pl        = rng.uniform(0.3, 1.5, npl) # semi-major axis
e_pl        = rng.uniform(0.0, 0.2, npl) # eccentricity
inc_pl      = rng.uniform(0.0, 10, npl) # inclination (degrees)
capom_pl    = rng.uniform(0.0, 360.0, npl) # longitude of the ascending node
omega_pl    = rng.uniform(0.0, 360.0, npl) # argument of periapsis
capm_pl     = rng.uniform(0.0, 360.0, npl) # mean anomaly

sim.add_body(name=name_pl, a=a_pl, e=e_pl, inc=inc_pl, capom=capom_pl, omega=omega_pl, capm=capm_pl, mass=M_pl, radius=R_pl,  Ip=Ip_pl, rot=rot_pl)

Adding bodies: SemiBody_01, SemiBody_02, SemiBody_03, SemiBody_04, SemiBody_05
rmin                             0.004650467260962157 DU
Writing initial conditions to file /Users/daminton/work/Talks/2025-10-TAP_University_of_Arizona/Swiftest_2025_TAP_Workshop/detailed_simulation/tap_example/init_cond.nc
Writing parameter inputs to file /Users/daminton/work/Talks/2025-10-TAP_University_of_Arizona/Swiftest_2025_TAP_Workshop/detailed_simulation/tap_example/param.in


The process is similar for **test particles**. They only need the orbital elements or the cartesian coordinates. 
Here is an example with orbital elements:

In [8]:
# Add 10 user-defined test particles.
ntp = 10

name_tp     = ["TestParticle_01", "TestParticle_02", "TestParticle_03", "TestParticle_04", "TestParticle_05", "TestParticle_06", "TestParticle_07", "TestParticle_08", "TestParticle_09", "TestParticle_10"]
a_tp        = rng.uniform(0.3, 1.5, ntp)
e_tp        = rng.uniform(0.0, 0.2, ntp)
inc_tp      = rng.uniform(0.0, 10, ntp)
capom_tp    = rng.uniform(0.0, 360.0, ntp)
omega_tp    = rng.uniform(0.0, 360.0, ntp)
capm_tp     = rng.uniform(0.0, 360.0, ntp)

sim.add_body(name=name_tp, a=a_tp, e=e_tp, inc=inc_tp, capom=capom_tp, omega=omega_tp, capm=capm_tp)

Adding bodies: TestParticle_01, TestParticle_02, TestParticle_03, TestParticle_04, TestParticle_05, TestParticle_06, TestParticle_07, TestParticle_08, TestParticle_09, TestParticle_10
rmin                             0.004650467260962157 DU
Writing initial conditions to file /Users/daminton/work/Talks/2025-10-TAP_University_of_Arizona/Swiftest_2025_TAP_Workshop/detailed_simulation/tap_example/init_cond.nc
Writing parameter inputs to file /Users/daminton/work/Talks/2025-10-TAP_University_of_Arizona/Swiftest_2025_TAP_Workshop/detailed_simulation/tap_example/param.in


In [19]:
sim.data

<xarray.SwiftestDataset> Size: 4kB
Dimensions:        (time: 1, name: 3, space: 3, sign: 2, l: 7, m: 7)
Coordinates:
  * time           (time) float64 8B 0.0
  * name           (name) <U32 384B 'Mars' 'Phobos' 'Deimos'
  * space          (space) <U1 12B 'x' 'y' 'z'
  * sign           (sign) float64 16B 1.0 -1.0
  * l              (l) float64 56B 0.0 1.0 2.0 3.0 4.0 5.0 6.0
  * m              (m) float64 56B 0.0 1.0 2.0 3.0 4.0 5.0 6.0
Data variables:
    a              (time, name) float64 24B nan 6.27e-05 0.0001568
    e              (time, name) float64 24B nan 0.01543 0.0003124
    inc            (time, name) float64 24B nan 26.56 24.06
    capom          (time, name) float64 24B nan 85.23 81.82
    omega          (time, name) float64 24B nan 7.167 48.29
    capm           (time, name) float64 24B nan 41.12 68.69
    varpi          (time, name) float64 24B nan 92.4 130.1
    lam            (time, name) float64 24B nan 133.5 198.8
    f              (time, name) float64 24B nan 42.3 68.73
    cape           (time, name) float64 24B nan 41.71 68.71
    capf           (time, name) float64 24B nan 0.0 0.0
    Gmass          (time, name) float64 24B 1.274e-05 2.144e-13 3.574e-14
    mass           (time, name) float64 24B 3.227e-07 5.431e-15 9.052e-16
    radius         (time, name) float64 24B 2.266e-05 7.464e-08 4.211e-08
    rhill          (time, name) float64 24B 0.007247 2.102e-05 1.138e-05
    rotphase       (time, name) float64 24B 0.0 0.0 0.0
    j2rp2          (time, name) float64 24B nan nan nan
    j4rp4          (time, name) float64 24B nan nan nan
    rh             (time, name, space) float64 72B 0.0 0.0 ... 5.696e-05
    vh             (time, name, space) float64 72B 0.0 0.0 ... -0.2682 -0.05277
    rot            (time, name, space) float64 72B 5.718e+04 ... 9.506e+04
    Ip             (time, name, space) float64 72B 0.0 0.0 0.3644 ... 0.4 0.4
    id             (name) int64 24B 0 1 2
    particle_type  (name) <U32 384B 'Central Body' 'Massive Body' 'Massive Body'
    status         (time, name) int64 24B 0 0 0
    ntp            (time) int64 8B 0
    npl            (time) int64 8B 2
    nplm           (time) int64 8B 2
    c_lm           (time, name, sign, l, m) float64 2kB 1.0 0.0 0.0 ... nan nan

## Customising Simulation Parameters


Now that we have added the bodies, we can set the simulation parameters. ``tstop`` and ``dt`` need to be set before running the simulation.
This can be done in multiple ways:

- When creating the initial Swiftest simulation object

```python
    sim = swiftest.Simulation(integrator = 'symba', tstart=0.0, tstop=1.0e3, dt=0.01, 
                                tstep_out=1.0, dump_cadence=0, compute_conservation_values=True, mtiny=mtiny)
```   
- `sim.set_parameter`: Set individual parameters in the simulation. The user can set one or multiple at a time.

In [9]:
sim.set_parameter(tstart=0.0, tstop=1.0e3, dt=0.01, tstep_out=1.0, dump_cadence=0, compute_conservation_values=True, mtiny=mtiny)
sim.set_parameter(rmin = 0.05)

codename                         Swiftest
gmtiny                           1.0919433586029915e-07 DU^3 / TU^2 
mtiny                            2.766029318728523e-09 MU
tstart                           0.0 TU
tstop                            1000.0 TU
dt                               0.01 TU
istep_out                        100 
dump_cadence                     0 
tstep_out                        1.0 TU
compute_conservation_values      True
codename                         Swiftest
rmin                             0.05 DU


{'CHK_RMIN': 0.05, 'CHK_QMIN': 0.05, 'CHK_QMIN_RANGE': '0.05 10000.0'}

We now set up the simulation parameters. Here we have a simulation starting from `0.0 y` and running for `1 My = 1e6 years` 
with time steps of `0.01 years`. The timestep should be less than or equal to 1/20 of the orbital period of the innermost body. 

The user can then write the parameters to the `param.in` file by using :meth:`write_param <swiftest.Simulation.write_param>`.
To see the parameters of the simulation, use :meth:`sim.get_parameter <swiftest.Simulation.get_parameter>`.

## Running the Simulation


Once everything is set up, we can save the simulation object and then run it.

In [10]:
sim.run()

Cleaning up simulation directory /Users/daminton/work/Talks/2025-10-TAP_University_of_Arizona/Swiftest_2025_TAP_Workshop/detailed_simulation/tap_example of old output files.
Writing initial conditions to file /Users/daminton/work/Talks/2025-10-TAP_University_of_Arizona/Swiftest_2025_TAP_Workshop/detailed_simulation/tap_example/init_cond.nc
Writing parameter inputs to file /Users/daminton/work/Talks/2025-10-TAP_University_of_Arizona/Swiftest_2025_TAP_Workshop/detailed_simulation/tap_example/param.in
Writing parameter inputs to file /Users/daminton/work/Talks/2025-10-TAP_University_of_Arizona/Swiftest_2025_TAP_Workshop/detailed_simulation/tap_example/param.in
Running a Swiftest symba run from tstart=0.0 TU to tstop=1000.0 TU
[                                                                                ] Time =  0.00000E+00 of  1.00000E+03 OpenMP: Number of threads =   1
[################################################################################] Time =  1.00000E+03 of  1.00000E+

Once this is finished, you should be able to access the output data stored in the `swiftest.Simulation.data` attribute.

In [14]:
# Import xarray and set its output to show more lines

sim.data

<xarray.SwiftestDataset> Size: 4kB
Dimensions:        (time: 1, name: 3, space: 3, sign: 2, l: 7, m: 7)
Coordinates:
  * time           (time) float64 8B 0.0
  * name           (name) <U32 384B 'Mars' 'Phobos' 'Deimos'
  * space          (space) <U1 12B 'x' 'y' 'z'
  * sign           (sign) float64 16B 1.0 -1.0
  * l              (l) float64 56B 0.0 1.0 2.0 3.0 4.0 5.0 6.0
  * m              (m) float64 56B 0.0 1.0 2.0 3.0 4.0 5.0 6.0
Data variables:
    a              (time, name) float64 24B nan 6.27e-05 0.0001568
    e              (time, name) float64 24B nan 0.01543 0.0003124
    inc            (time, name) float64 24B nan 26.56 24.06
    capom          (time, name) float64 24B nan 85.23 81.82
    omega          (time, name) float64 24B nan 7.167 48.29
    capm           (time, name) float64 24B nan 41.12 68.69
    varpi          (time, name) float64 24B nan 92.4 130.1
    lam            (time, name) float64 24B nan 133.5 198.8
    f              (time, name) float64 24B nan 42.3 68.73
    cape           (time, name) float64 24B nan 41.71 68.71
    capf           (time, name) float64 24B nan 0.0 0.0
    Gmass          (time, name) float64 24B 1.274e-05 2.144e-13 3.574e-14
    mass           (time, name) float64 24B 3.227e-07 5.431e-15 9.052e-16
    radius         (time, name) float64 24B 2.266e-05 7.464e-08 4.211e-08
    rhill          (time, name) float64 24B 0.007247 2.102e-05 1.138e-05
    rotphase       (time, name) float64 24B 0.0 0.0 0.0
    j2rp2          (time, name) float64 24B nan nan nan
    j4rp4          (time, name) float64 24B nan nan nan
    rh             (time, name, space) float64 72B 0.0 0.0 ... 5.696e-05
    vh             (time, name, space) float64 72B 0.0 0.0 ... -0.2682 -0.05277
    rot            (time, name, space) float64 72B 5.718e+04 ... 9.506e+04
    Ip             (time, name, space) float64 72B 0.0 0.0 0.3644 ... 0.4 0.4
    id             (name) int64 24B 0 1 2
    particle_type  (name) <U32 384B 'Central Body' 'Massive Body' 'Massive Body'
    status         (time, name) int64 24B 0 0 0
    ntp            (time) int64 8B 0
    npl            (time) int64 8B 2
    nplm           (time) int64 8B 2
    c_lm           (time, name, sign, l, m) float64 2kB 1.0 0.0 0.0 ... nan nan

Or, say, plot the eccentricity history of just the test particles:

In [16]:
etp=sim.data.e.where(sim.data.particle_type == 'Test Particle',drop=True)#.plot(x='time',hue='name');

In [18]:
sim.data.particle_type

<xarray.SwiftestDataArray 'particle_type' (name: 3)> Size: 384B
array(['Central Body', 'Massive Body', 'Massive Body'], dtype='<U32')
Coordinates:
  * name     (name) <U32 384B 'Mars' 'Phobos' 'Deimos'